In [28]:
import tensorflow as tf
import tensorflow.keras
from homemade_unet import unet_model_3d
from unet_scratch import get_unet
import os
from data import get_train_and_validation_datasets
import nibabel as nib

In [13]:
#from google.datalab.ml import TensorBoard
PROJECT = 'freelearing'
BUCKET = 'postgraduate_ai'
REGION = 'europe-west4' 

In [14]:
# for bash
os.environ['PROJECT'] = PROJECT
os.environ['BUCKET'] = BUCKET
os.environ['REGION'] = REGION
os.environ['TFVERSION'] = '2.1'  # Tensorflow version
os.environ['PYTHONVERSION'] = '3.7' # Python version - 3.7 can only be used with TF Version 1.15 or higher

In [17]:
input_shape = (1,64,64,64)
model = unet_model_3d(input_shape = input_shape, n_labels = 3)
#model = get_unet(input_img = input_shape, n_labels = 3, n_filters = 16, batchnorm = True)
#model.summary()

In [18]:
batch_size = 8
#path_images = '/home/jupyter/ai_postgraduate_project/data/raw_dataset/imagesTr/'
#path_labels = '/home/jupyter/ai_postgraduate_project/data/raw_dataset/labelsTr/'

path_images = 'gs://postgraduate_ai/resized_dataset/imagesTr/'
path_labels = 'gs://postgraduate_ai/resized_dataset/labelsTr/'

In [19]:
train_dataset, validation_dataset, validation_images = get_train_and_validation_datasets(
    0.2,
    path_images,
    path_labels,
    is_data_cloud = True,
    patch=True,
    patch_shape=(64,64,64),
    subsample=20
)

Subsampling with images: ['pancreas_001.nii.gz', 'pancreas_004.nii.gz', 'pancreas_005.nii.gz', 'pancreas_006.nii.gz', 'pancreas_010.nii.gz', 'pancreas_012.nii.gz', 'pancreas_015.nii.gz', 'pancreas_016.nii.gz', 'pancreas_018.nii.gz', 'pancreas_019.nii.gz', 'pancreas_021.nii.gz', 'pancreas_024.nii.gz', 'pancreas_025.nii.gz', 'pancreas_028.nii.gz', 'pancreas_029.nii.gz', 'pancreas_032.nii.gz', 'pancreas_035.nii.gz', 'pancreas_037.nii.gz', 'pancreas_040.nii.gz', 'pancreas_041.nii.gz']
Number of images for training: 16
Number of images for validation: 4


In [20]:
train_dataset = train_dataset.shuffle(100).batch(batch_size).prefetch(2)
validation_dataset = validation_dataset.batch(batch_size).prefetch(2)

In [26]:
test_image = '/home/jupyter/ai_postgraduate_project/data/raw_dataset/imagesTr/pancreas_004.nii.gz'
img = nib.load(test_image)
print(img.shape)

(512, 512, 107)


In [30]:
test_image2 = 'gs://postgraduate_ai/resized_dataset/imagesTr/pancreas_004.nii.gz'
print(tf.data.TextLineDataset(test_image2))

tf.data.from_tensor_slices([8, 3, 0, 8, 2, 1])

<TextLineDatasetV2 shapes: (), types: tf.string>


In [ ]:
for image in train_dataset:
    x,y = image
    print(x.shape)
    print(y.shape)
    break

In [4]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, ReduceLROnPlateau
from datetime import datetime

time = datetime.now().strftime('%Y-%m-%d_%H-%M')

model_checkpoint = ModelCheckpoint(
    filepath='/home/jupyter/ai_postgraduate_project/notebooks/cpachon/models/best/' + time + '_model_weights.h5',
    monitor='val_loss',
    verbose=1,
    save_best_only=True)

reduce_lr_plateau = ReduceLROnPlateau(monitor='val_loss', 
                                      patience=10, 
                                      verbose=1)

tensorboard_callback = TensorBoard(log_dir='runs/' + time,
                                   write_grads=True)

callbacks = [model_checkpoint,reduce_lr_plateau,tensorboard_callback]


In [5]:
epochs = 80
model.fit(train_dataset,
          epochs=epochs,
          validation_data=validation_dataset,
          callbacks = callbacks)

Epoch 1/80
     60/Unknown - 272s 5s/step - loss: 2.4260
Epoch 00001: val_loss improved from inf to 0.92509, saving model to /home/jupyter/ai_postgraduate_project/notebooks/cpachon/models/best/2020-03-14_18-19_model_weights.h5
60/60 [==============================] - 298s 5s/step - loss: 2.4260 - val_loss: 0.9251
Epoch 2/80
46/60 [======================>.......] - ETA: 59s - loss: 1.5368 WARNING:tensorflow:Can save best model only with val_loss available, skipping.


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-5-d90bf0ec6193>", line 5, in <module>
    callbacks = callbacks)
  File "/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/training.py", line 819, in fit
    use_multiprocessing=use_multiprocessing)
  File "/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/training_v2.py", line 342, in fit
    total_epochs=epochs)
  File "/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/training_v2.py", line 128, in run_one_epoch
    batch_outs = execution_function(iterator)
  File "/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/training_v2_utils.py", line 98, in execution_function
    distributed_function(input_fn))
  File "/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/u

KeyboardInterrupt: 

In [ ]:
model.save_weights("models/final/" + time + "_model_weights.h5")